In [1]:
%display latex
from pylie import *
from pprint import pprint 
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint
from IPython.core.debugger import set_trace


In [2]:
def reduce(e1: Differential_Polynomial,e2: Differential_Polynomial, context:Context)->Differential_Polynomial:
    assert e2.is_monic()
    def _order (der):
        if der != 1:
            ## XXX: user pylie namespace
            return order_of_derivative(der)
        else :
            return [0]*len(context._independent)
    def func(e):    
        try:
            return e.operator().function()
        except AttributeError:
            return e.operator()   

    def _reduce (e, ld):
        e2_order = _order (ld)
        for t in e._p:
            d = t._d
            c = t._coeff
            if func(ld) != func(d):
                continue
            e1_order = _order (d)
            dif = [a-b for a, b in zip (e1_order, e2_order)]
            if all (map (lambda h: h == 0, dif)) :
                return Differential_Polynomial (e1.expression() - e2.expression() * c, context)
            elif all (map (lambda h: h >= 0, dif)):         
                variables_to_diff = []
                for i in range (len(context._independent)):
                    if dif [i] != 0:
                        variables_to_diff.extend ([context._independent[i]]*abs(dif[i]))      
                s  = sum([c * _._coeff * diff(_._d,  *variables_to_diff) for _ in e2._p])
                return Differential_Polynomial (e1.expression()-s, context)
            else:
                pass
        return e

    _e1 = None
    while True:
        _e1 = _reduce (e1, e2.Lder())        
        if _e1 == e1:
            return e1
        e1 = _e1
        

In [3]:
#indeterminates = (x, y) = var("x y")
#R=PolynomialRing(QQ, indeterminates)
#z  = function ("z")(*indeterminates)
#ctx = Context((z,), indeterminates)
#print("*"*70)
#e1 = diff(z,y)-x**2/y**2*diff(z,x)-(x-y)*z(x,y)/y**2
#e1_dp = Differential_Polynomial (e1, ctx)
#print (e1_dp)
#print("*"*70)
#f1 = diff(z, x) + z(x,y)/x
#f1_dp = Differential_Polynomial (f1, ctx)
#f1_dp.is_monic()
#print (f1_dp)
#print("*"*70)
#f2 = diff(z, y) + z(x,y)/y
#f2_dp = Differential_Polynomial (f2, ctx)
#print (f2_dp)
#r1 = reduce (e1_dp, f1_dp, ctx)
#r1
#r1._orig
#r2 = reduce (r1, f2_dp, ctx)

In [4]:
def Lfunc(e):
    return e

In [5]:
def Lcml(e):
    '''Least common left multiply, p.23'''
    return 0

In [6]:
def Lcm(e):
    return 0

In [7]:
def CompleteSystem(S, context):
    return S

In [8]:
def FindIntegrableConditions(S, context):
    return S

In [9]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [10]:

def Reorder (S, context, ascending = False):
    res=sorted(S)
    if ascending :
        res.reverse()
    return res

In [11]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
ctx = Context((w,z), (x,y), MatrixOrder.Mgrlex)
f1 =diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w(x,y)/y
f1.show()

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [12]:
f2=diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
f2.show()

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

In [13]:
f3 = diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2
f3.show()

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

In [14]:
f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w(x,y)/(2*y**2)
f4.show()

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

In [15]:
f5 =diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w(x,y)/(y**2)
f5.show()

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [16]:
#f2a=f2-(y*f1/x);f2a.expand().show()

In [17]:
f1dp=Differential_Polynomial(f1, context=ctx)
f2dp=Differential_Polynomial(f2, context=ctx)
f3dp=Differential_Polynomial(f3, context=ctx)
f4dp=Differential_Polynomial(f4, context=ctx)
f5dp=Differential_Polynomial(f5, context=ctx)

In [18]:
f1.show()
f2.show()
cf2a=reduce(f2dp, f1dp, ctx)
cf2a.show()

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

2*y*diff(z(x, y), x)/x + w(x, y)/x - 1/2*diff(z(x, y), y)/(x^2 + y) + diff(z(x, y), x, y)

In [19]:
(f2 - diff(f1, y) * y/x).expand().show()
cf=Differential_Polynomial((f2 - diff(f1, y) * y/x).expand(),ctx)
cf.show()

y*diff(w(x, y), y)/x - y*diff(w(x, y), y, y)/x + 2*y*diff(z(x, y), x)/x + diff(w(x, y), y)/x + 1/2*diff(z(x, y), y)/(x^4 + 2*x^2*y + y^2) - 1/2*diff(z(x, y), y, y)/(x^2 + y) - w(x, y)/(x*y) + 1/2*diff(z(x, y), y)/((x^2 + y)*y) + diff(z(x, y), x, y)

-(y + 1)*diff(w(x, y), y)/y - x*diff(z(x, y), x, y)/y - 1/2*(x^3 + 2*x*y)*diff(z(x, y), y)/(x^4*y^2 + 2*x^2*y^3 + y^4) + 1/2*x*diff(z(x, y), y, y)/(x^2*y + y^2) + w(x, y)/y^2 + diff(w(x, y), y, y) - 2*diff(z(x, y), x)

In [20]:
diff (f1,y).collect(diff(z, y))

-1/2*(x/((x^2 + y)*y^2) + x/((x^2 + y)^2*y))*diff(z(x, y), y) - diff(w(x, y), y)/y + 1/2*x*diff(z(x, y), y, y)/((x^2 + y)*y) + w(x, y)/y^2 + diff(w(x, y), y, y)

In [21]:
(f2 - diff(f1, y) * y/x).expand()
cf=Differential_Polynomial((f2 - diff(f1, y) * y/x).expand(),ctx)
cf.show()

-(y + 1)*diff(w(x, y), y)/y - x*diff(z(x, y), x, y)/y - 1/2*(x^3 + 2*x*y)*diff(z(x, y), y)/(x^4*y^2 + 2*x^2*y^3 + y^4) + 1/2*x*diff(z(x, y), y, y)/(x^2*y + y^2) + w(x, y)/y^2 + diff(w(x, y), y, y) - 2*diff(z(x, y), x)

In [22]:
cf2a.expression().expand().collect(diff(w,y)).collect(w(x,y)).show()

2*y*diff(z(x, y), x)/x + w(x, y)/x - 1/2*diff(z(x, y), y)/(x^2 + y) + diff(z(x, y), x, y)

In [23]:
def Autoreduce(S, context):  
    i = 0
    while True:
        p = S[i]
        r = S[i+1:]
        if not r:
            S = Reorder(S, context, ascending = True)
            return S[:]  
        #set_trace ()
        print ("=====================================================", len(r))
        have_reduced = False
        #set_trace()
        checkidx = i+1
        for _r in r:
            #print ("Try to reduce {} with {}".format(S[checkidx], S[i].Lder()))
            res = reduce (_r, p, context)
            if res != _r:
                have_reduced = True                
            S[checkidx] = res
            checkidx += 1
        #print("This is the new System before reorder")   
        #for _ in S:          
        #    _.show()
        S = Reorder(S, context, ascending = True)
        print("This is the new System........")
        for _ in S:          
            _.show()
        if not have_reduced:
            # no reduction done
            i += 1        
        else:
            i = 0
            

In [24]:
def JanetBasis(S, context):
    """List of homogenous PDE's + order context"""
    if not isinstance(S, Iterable):
        # bad criterion
        S = [S]
    S = [Differential_Polynomial(s, context) for s in S]
    for _ in S:
        _.show()
    print ("....................................")
    S = Reorder(S, context, ascending = True)
    for _ in S:
        _.show()
    while 1:
        S = Autoreduce (S, context)
        #S = CompleteSystem (S, context)
        #f = FindIntegrableConditions(S, context)
        #c = ReduceIntegrabilityConditions(S, context)
        #if all ([_ == 0 for _ in c]) or not c:
        #    return S
        #S += c
        #S = Reorder (S, context)
        return S[:]

In [25]:
checkS=JanetBasis([f3,f1,f5, f2,f4], ctx)

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

1/2*x*diff(z(x, y), y)/(x^2*y + y^2) - w(x, y)/y + diff(w(x, y), y)

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

...

-1/4*(x^10 + 4*(4*x^6 + 4*x^5 + x^4)*y^6 + 8*(8*x^8 + 6*x^7 + x^6)*y^5 + 4*(24*x^10 + 12*x^9 + x^8 - 4*x^6 - 2*x^5)*y^4 + 4*(16*x^12 + 4*x^11 - 10*x^8 - 3*x^7)*y^3 + 4*(4*x^14 - 8*x^10 - x^9 + x^6)*y^2 - 4*(2*x^12 - x^8)*y)*w(x, y)/(16*x^2*y^10 + x^6*y^5 + 4*y^11 + 4*(6*x^4 - x)*y^9 + 4*(4*x^6 - 3*x^3)*y^8 + (4*x^8 - 12*x^5 + x^2)*y^7 - 2*(2*x^7 - x^4)*y^6) - 1/2*(x^9 - 2*(2*x^7 + x^6)*y^3 - 2*(4*x^9 + x^8)*y^2 - 2*(2*x^11 - x^7)*y)*w(x, y)/(16*x^2*y^8 + x^6*y^3 + 4*y^9 + 4*(6*x^4 - x)*y^7 + 4*(4*x^6 - 3*x^3)*y^6 + (4*x^8 - 12*x^5 + x^2)*y^5 - 2*(2*x^7 - x^4)*y^4) - 1/4*(x^8 + 4*(4*x^4 + 4*x^3 + x^2)*y^6 + 8*(8*x^6 + 6*x^5 + x^4)*y^5 + 4*(24*x^8 + 12*x^7 + x^6 - 4*x^4 - 2*x^3)*y^4 + 4*(16*x^10 + 4*x^9 - 10*x^6 - 3*x^5)*y^3 + 4*(4*x^12 - 8*x^8 - x^7 + x^4)*y^2 - 4*(2*x^10 - x^6)*y)*w(x, y)/(16*x^2*y^9 + x^6*y^4 + 4*y^10 + 4*(6*x^4 - x)*y^8 + 4*(4*x^6 - 3*x^3)*y^7 + (4*x^8 - 12*x^5 + x^2)*y^6 - 2*(2*x^7 - x^4)*y^5) + 1/4*(x^8 - 2*(2*x^6 + x^5)*y^3 - 2*(4*x^8 + x^7)*y^2 - 2*(2*x^10 - x^6)*y)*w(x, y)/(16*x^2*y^9 + x^6*y^4 + 4*y^10 + 4*(6*x^4 - x)*y^8 + 4*(4*x^6 - 3*x^3)*y^7 + (4*x^8 - 12*x^5 + x^2)*y^6 - 2*(2*x^7 - x^4)*y^5) - (x^7 - 2*(2*x^5 + x^4)*y^3 - 2*(4*x^7 + x^6)*y^2 - 2*(2*x^9 - x^5)*y)*w(x, y)/(16*x^2*y^7 + x^6*y^2 + 4*y^8 + 4*(6*x^4 - x)*y^6 + 4*(4*x^6 - 3*x^3)*y^5 + (4*x^8 - 12*x^5 + x^2)*y^4 - 2*(2*x^7 - x^4)*y^3) + 1/4*(x^6 - 2*(2*x^4 + x^3)*y^3 - 2*(4*x^6 + x^5)*y^2 - 2*(2*x^8 - x^4)*y)*w(x, y)/(16*x^2*y^8 + x^6*y^3 + 4*y^9 + 4*(6*x^4 - x)*y^7 + 4*(4*x^6 - 3*x^3)*y^6 + (4*x^8 - 12*x^5 + x^2)*y^5 - 2*(2*x^7 - x^4)*y^4) - 1/2*(x^5 - 2*(2*x^3 + x^2)*y^3 - 2*(4*x^5 + x^4)*y^2 - 2*(2*x^7 - x^3)*y)*w(x, y)/(16*x^2*y^6 + x^6*y + 4*y^7 + 4*(6*x^4 - x)*y^5 + 4*(4*x^6 - 3*x^3)*y^4 + (4*x^8 - 12*x^5 + x^2)*y^3 - 2*(2*x^7 - x^4)*y^2) + 1/8*(32*x^2*y^10 - x^11 + 16*(4*x^5 + 12*x^4 + x^3)*y^9 + 8*(48*x^7 + 64*x^6 + 8*x^5 - 4*x^3 - x^2)*y^8 + x^8 + 4*(240*x^9 + 192*x^8 + 24*x^7 - 44*x^5 - 22*x^4 - 3*x^3)*y^7 + 4*(320*x^11 + 168*x^10 + 16*x^9 - 116*x^7 - 66*x^6 - 9*x^5 + 2*x^4 + 2*x^3)*y^6 + 4*(240*x^13 + 80*x^12 + 4*x^11 - 176*x^9 - 86*x^8 - 9*x^7 + 4*x^6 + 16*x^5 + 6*x^4)*y^5 + 2*(192*x^15 + 32*x^14 - 304*x^11 - 104*x^10 - 6*x^9 + 4*x^8 + 80*x^7 + 28*x^6 - x^4)*y^4 + 2*(32*x^17 - 136*x^13 - 24*x^12 + 86*x^9 + 20*x^8 - 3*x^6 - 7*x^5)*y^3 - 2*(24*x^15 - 40*x^11 - 4*x^10 + 4*x^8 + 11*x^7)*y^2 + (12*x^13 - 4*x^10 - 9*x^9 + 2*x^6)*y)*diff(z(x, y), y)/(40*x^2*y^11 + 8*y^12 - x^7*y^4 + 4*(20*x^4 - 3*x)*y^10 + 16*(5*x^6 - 3*x^3)*y^9 + 2*(20*x^8 - 36*x^5 + 3*x^2)*y^8 + 2*(4*x^10 - 24*x^7 + 9*x^4)*y^7 - (12*x^9 - 18*x^6 + x^3)*y^6 + 2*(3*x^8 - x^5)*y^5) + diff(w(x, y), x)

1/4*(8*x^2*y^6 + x^6 + 8*(4*x^4 + x^3)*y^5 + 2*(28*x^6 + 8*x^5 - 2*x^3 - x^2)*y^4 + 2*(24*x^8 + 4*x^7 - 2*x^5 - 7*x^4 - x^3)*y^3 + (16*x^10 - 20*x^6 - 2*x^5 + x^3)*y^2 - 2*(4*x^8 - x^4)*y)*diff(z(x, y), y)/(12*x^2*y^7 + 4*y^8 + x^4*y^3 + 4*(3*x^4 - x)*y^6 + 4*(x^6 - 2*x^3)*y^5 - (4*x^5 - x^2)*y^4) + diff(w(x, y), x)

-4*x^3*y*w(x, y)/(2*(2*x + 1)*y^3 - x^3 + 2*(4*x^3 + x^2)*y^2 + 2*(2*x^5 - x)*y) - 8*x*y^2*w(x, y)/(2*(2*x + 1)*y^3 - x^3 + 2*(4*x^3 + x^2)*y^2 + 2*(2*x^5 - x)*y) + 4*y^3*w(x, y)/(x^4 - 2*(2*x^2 + x)*y^3 - 2*(4*x^4 + x^3)*y^2 - 2*(2*x^6 - x^2)*y) + 2*x^2*w(x, y)/(2*(2*x + 1)*y^3 - x^3 + 2*(4*x^3 + x^2)*y^2 + 2*(2*x^5 - x)*y) + 2*y*w(x, y)/(2*(2*x + 1)*y^3 - x^3 + 2*(4*x^3 + x^2)*y^2 + 2*(2*x^5 - x)*y) - 8*(16*x^2*y^9 + x^6*y^4 + 4*y^10 + 4*(6*x^4 - x)*y^8 + 4*(4*x^6 - 3*x^3)*y^7 + (4*x^8 - 12*x^5 + x^2)*y^6 - 2*(2*x^7 - x^4)*y^5)*diff(w(x, y), x)/(x^9 + 4*(4*x^5 + 4*x^4 + x^3)*y^6 + 8*(8*x^7 + 6*x^6 + x^5)*y^5 + 4*(24*x^9 + 12*x^8 + x^7 - 4*x^5 - 2*x^4)*y^4 + 4*(16*x^11 + 4*x^10 - 10*x^7 - 3*x^6)*y^3 + 4*(4*x^13 - 8*x^9 - x^8 + x^5)*y^2 - 4*(2*x^11 - x^7)*y) - (48*x^2*y^7 + 16*y^8 + 4*(12*x^4 - 2*x - 1)*y^6 + 4*(4*x^6 - 4*x^3 - 4*x^2 - x)*y^5 - x^5 - 2*(4*x^5 + 14*x^4 + 4*x^3 - x)*y^4 - 2*(12*x^6 + 2*x^5 - 3*x^3 - 3*x^2)*y^3 - 8*(x^8 - x^5 - x^4)*y^2 + 2*(2*x^7 + x^6 - x^3)*y)*diff(z(x, y), y)/(x^7 + 4*(4*x^3 + 4*x^2 + x)*y^6 + 8*(8*x^5 + 6*x^4 + x^3)*y^5 + 4*(24*x^7 + 12*x^6 + x^5 - 4*x^3 - 2*x^2)*y^4 + 4*(16*x^9 + 4*x^8 - 10*x^5 - 3*x^4)*y^3 + 4*(4*x^11 - 8*x^7 - x^6 + x^3)*y^2 - 4*(2*x^9 - x^5)*y) + diff(z(x, y), y, y)

===================================================== 2


RuntimeError: ECL says: Console interrupt.

In [ ]:
for _ in checkS:
    _.show()

In [ ]:
g1 = Differential_Polynomial (diff(z,y,y) + diff(z,y)/(2*y), ctx)

In [ ]:
g2 = Differential_Polynomial(diff(w,x,x) + 4*y*y*diff(w,y) - 8*y*y*diff(z, x) - 8*y*w(x,y), ctx)

In [ ]:
g3 = Differential_Polynomial(diff(w,x,y) - diff(z,x,x)/2 - diff(w,x)/(2*y) - 6*y**2*diff(z,y), ctx)

In [ ]:
g4 = Differential_Polynomial(diff(w,y,y) - 2*diff(z,x,y)- diff(w,y)/(2*y) + w(x,y)/(2*y**2), ctx)